# Deploy model
**Important**: Change the kernel to *PROJECT_NAME local*. You can do this from the *Kernel* menu under *Change kernel*. You cannot deploy the model using the *PROJECT_NAME docker* kernel.

In [1]:
from azureml.api.schema.dataTypes import DataTypes
from azureml.api.schema.sampleDefinition import SampleDefinition
from azureml.api.realtime.services import generate_schema
import pandas as pd
import numpy as np
import imp
import pickle
import os
import sys
import json

In [2]:
from azureml.logging import get_azureml_logger
run_logger = get_azureml_logger()
run_logger.log('amlrealworld.timeseries.deploy-model','true')

Enter the name of the model to deploy.

In [3]:
model_name = "dtree"

Load the test dataset and retain just one row. This record will be used to create and input schema for the web service. It will also allow us to simulate invoking the web service with features for one hour period and generating a demand forecast for this hour.

In [4]:
aml_dir = os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']
test_df = pd.read_csv(os.path.join(aml_dir, 'nyc_demand_test.csv'), parse_dates=['timeStamp'])
test_df = test_df.drop(['demand', 'timeStamp'], axis=1).copy().iloc[[0]]
test_df

,precip,temp,hour,month,dayofweek,temp_lag1,temp_lag2,temp_lag3,temp_lag4,temp_lag5,temp_lag6,demand_lag1,demand_lag2,demand_lag3,demand_lag4,demand_lag5,demand_lag6
0,0.0,74.63,0,6,4,75.1,75.72,76.72,75.85,77.36,80.92,6912.7,7332.625,7576.558,7603.008,7788.292,8102.142


Load model from disk and transfer it to the working directory. 

In [21]:
model_dir = "C:/Users/nelgoh/Desktop/Resources/Petronas/energy_demand_forecast/EnergyDemandForecast/models/"
with open(os.path.join(model_dir, model_name + '.pkl'), 'rb') as f:
    mod = pickle.load(f)

# with open('model_deploy.pkl', 'wb') as f:
#     pickle.dump(mod, f)

Check model object has loaded as expected.

In [22]:
mod

Pipeline(steps=[('regr_cv', RandomizedSearchCV(cv=TimeSeriesSplit(n_splits=3), error_score='raise',
          estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
     ...it=True,
          return_train_score=True, scoring='neg_mean_squared_error',
          verbose=2))])

Apply model to predict test record

In [23]:
np.asscalar(mod.predict(test_df))

6431.91218181818

### Author a realtime web service

Create a score.py script which implements the scoring function to run inside the web service. Change model_name variable as required.

In [40]:
%%writefile C:/Users/nelgoh/Desktop/Resources/Petronas/energy_demand_forecast/EnergyDemandForecast/score.py
# The init and run functions will load and score the input using the saved model.
# The score.py file will be included in the web service deployment package.
def init():
    from sklearn.externals import joblib
    import pickle
    import os
    global model
    
    model_dir = "./models/dtree.pkl"
    model = joblib.load(model_dir)
#     model_dir = "C:/Users/nelgoh/Desktop/Resources/Petronas/energy_demand_forecast/EnergyDemandForecast/models/"
#     with open(os.path.join(model_dir, model_name + '.pkl'), 'rb') as f:
#         model = pickle.load(f)
#     with open('model_deploy.pkl', 'rb') as f:
#         model = pickle.load(f)
    
def run(input_df):
    input_df = input_df[['precip', 'temp', 'hour', 'month', 'dayofweek',
        'temp_lag1', 'temp_lag2', 'temp_lag3', 'temp_lag4', 'temp_lag5',
        'temp_lag6', 'demand_lag1', 'demand_lag2', 'demand_lag3',
        'demand_lag4', 'demand_lag5', 'demand_lag6']]
    try:
        if (input_df.shape != (1,17)):
            return 'Bad imput: Expecting dataframe of shape (1,17)'
        else:
            pred = model.predict(input_df)
            return int(pred)
    except Exception as e:
        return(str(e))

Overwriting C:/Users/nelgoh/Desktop/Resources/Petronas/energy_demand_forecast/EnergyDemandForecast/score.py


This script will be written to the current working directory.

#### Test the *init* and *run* functions

In [25]:
import score
imp.reload(score)

<module 'score' from 'C:\\Users\\nelgoh\\AppData\\Local\\Temp\\4\\azureml_runs\\EnergyDemandForecast_1538106716523\\score.py'>

In [26]:
score.init()
score.run(test_df)

6431

#### Create web service schema
The web service schema provides details on the required structure of the input data as well as the data types of each column.

In [39]:
root_proj_dir = "C:/Users/nelgoh/Desktop/Resources/Petronas/energy_demand_forecast/EnergyDemandForecast/"
inputs = {"input_df": SampleDefinition(DataTypes.PANDAS, test_df)}
generate_schema(run_func=score.run, inputs=inputs, filepath=root_proj_dir + 'service_schema.json')

{'input': {'input_df': {'internal': 'gANjYXp1cmVtbC5hcGkuc2NoZW1hLnBhbmRhc1V0aWwKUGFuZGFzU2NoZW1hCnEAKYFxAX1xAihYDAAAAGNvbHVtbl90eXBlc3EDXXEEKGNudW1weQpkdHlwZQpxBVgCAAAAZjhxBksASwGHcQdScQgoSwNYAQAAADxxCU5OTkr/////Sv////9LAHRxCmJoCGgFWAIAAABpOHELSwBLAYdxDFJxDShLA2gJTk5OSv////9K/////0sAdHEOYmgNaA1oCGgIaAhoCGgIaAhoCGgIaAhoCGgIaAhlWAUAAABzaGFwZXEPSwFLEYZxEFgMAAAAY29sdW1uX25hbWVzcRFdcRIoWAYAAABwcmVjaXBxE1gEAAAAdGVtcHEUWAQAAABob3VycRVYBQAAAG1vbnRocRZYCQAAAGRheW9md2Vla3EXWAkAAAB0ZW1wX2xhZzFxGFgJAAAAdGVtcF9sYWcycRlYCQAAAHRlbXBfbGFnM3EaWAkAAAB0ZW1wX2xhZzRxG1gJAAAAdGVtcF9sYWc1cRxYCQAAAHRlbXBfbGFnNnEdWAsAAABkZW1hbmRfbGFnMXEeWAsAAABkZW1hbmRfbGFnMnEfWAsAAABkZW1hbmRfbGFnM3EgWAsAAABkZW1hbmRfbGFnNHEhWAsAAABkZW1hbmRfbGFnNXEiWAsAAABkZW1hbmRfbGFnNnEjZVgKAAAAc2NoZW1hX21hcHEkfXElKGgeaAhoF2gNaCBoCGgTaAhoHGgIaBtoCGghaAhoFmgNaBhoCGgaaAhoH2gIaB1oCGgiaAhoGWgIaCNoCGgVaA1oFGgIdXViLg==',
   'swagger': {'example': [{'dayofweek': 4,
      'demand_lag1': 6912.7,
      'demand_lag2': 7332.625,
      'demand_lag3': 757

#### Deploy the web service
The command below deploys a web service names "demandforecast", with input schema defined by "service_schema.json". The web service runs "score.py" which scores the input data using the model "model_deploy.pkl". This may take a few minutes.

In [30]:
!az ml env set -n modelstagingenv -g pet_cg_azure_demo

Compute set to modelstagingenv.


In [35]:
!az ml service create realtime -f score.py -m model_deploy.pkl -s service_schema.json -n demandforecast -r python

 model_deploy.pkl
Successfully registered model
Id: 2b7c1b6dd5b74996baf33bbdc034f4cb
More information: 'az ml model show -m 2b7c1b6dd5b74996baf33bbdc034f4cb'
Creating new driver at C:\Users\nelgoh\AppData\Local\Temp\4\tmpnxvk4xmw.py
 score.py
 service_schema.json
Successfully created manifest
Id: 7d1b5cd3-1da2-4651-93b5-8e983a5b2a57
More information: 'az ml manifest show -i 7d1b5cd3-1da2-4651-93b5-8e983a5b2a57'
Creating image.....Done.
Image ID: 0146874f-1622-492d-bd2a-b848b4a5bde3
More details: 'az ml image show -i 0146874f-1622-492d-bd2a-b848b4a5bde3'
Usage information: 'az ml image usage -i 0146874f-1622-492d-bd2a-b848b4a5bde3'
[Local mode] Running docker container.
[Local mode] Pulling the image from mlcrpacr1fb04691c711.azurecr.io/demandforecast:3. This may take a few minutes, depending on your connection speed...
[Local mode] Pulling....


ERROR: {'Error': MlCliError({'Error': 'Unable to start container', 'Response Content': APIError(HTTPError('500 Server Error: Internal Server Error for url: http+docker://localnpipe/v1.35/containers/a8b6e1422050512352df878f7cf0f70a26efb9964b44e94f692e219da93641c7/start',),)},), 'Azure-cli-ml Version': None}


Check web service is running.

In [34]:
!az ml service show realtime -i demandforecast

ERROR: {
    "Azure-cli-ml Version": null,
    "Error": "No service with id demandforecast is running locally."
}


Test the web service is working by invoking it with a test record.

In [ ]:
!az ml service run realtime -i demandforecast -d "{\"input_df\": [{\"hour\": 0, \"month\": 6, \"demand_lag3\": 7576.558, \"temp_lag5\": 77.36, \"temp\": 74.63, \"demand_lag1\": 6912.7, \"demand_lag5\": 7788.292, \"temp_lag6\": 80.92, \"temp_lag3\": 76.72, \"demand_lag6\": 8102.142, \"temp_lag4\": 75.85, \"precip\": 0.0, \"temp_lag2\": 75.72, \"demand_lag2\": 7332.625, \"temp_lag1\": 75.1, \"demand_lag4\": 7603.008, \"dayofweek\": 4}]}"

#### Delete the web service

In [ ]:
!az ml service delete realtime --id=demandforecast